# 📧 Clasificarea Spam-ului cu Naive Bayes
## Demonstrație Practică: De la Teoria Probabilității la AI Aplicat

**Obiective:**
- 🎯 Înțelegerea fundamentelor matematice ale Teoremei Bayes
- 🔧 Implementare de la zero a algoritmului Naive Bayes
- 📊 Vizualizarea rezultatelor și evaluarea performanței
- 💼 Aplicație practică pe date reale de spam

---

## 1️⃣ Fundamente Teoretice

### Teorema Bayes

Teorema lui Bayes, formulată în 1763, este fundamentul inferenței statistice:

$$P(H|E) = \frac{P(E|H) \times P(H)}{P(E)}$$

Unde:
- **P(H|E)** = Probabilitatea **posterioară** (ipoteza H dată evidența E)
- **P(E|H)** = **Likelihood** (probabilitatea evidenței E dacă H este adevărat)
- **P(H)** = Probabilitatea **priori** (credința inițială în H)
- **P(E)** = Probabilitatea evidenței (constantă de normalizare)

### Aplicație la Clasificarea Spam

Pentru clasificarea unui email cu cuvintele W₁, W₂, ..., Wₙ:

$$P(Spam|W_1, W_2, ..., W_n) = \frac{P(W_1, W_2, ..., W_n|Spam) \times P(Spam)}{P(W_1, W_2, ..., W_n)}$$

**Asumpția Naive:** Cuvintele sunt independente condiționat de clasă:

$$P(W_1, W_2, ..., W_n|Spam) = \prod_{i=1}^{n} P(W_i|Spam)$$

---

In [ ]:
# Importuri necesare
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict, Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import re
import warnings
warnings.filterwarnings('ignore')

# Configurare vizualizări
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✅ Biblioteci importate cu succes!")

## 2️⃣ Dataset de Spam Real

Vom crea un dataset realist de mesaje spam și ham (non-spam):

In [ ]:
# Dataset de exemple - mesaje spam și ham în română și engleză
spam_messages = [
    "FELICITĂRI! Ai câștigat 1000 EURO! Click aici ACUM pentru a revendica premiul!",
    "Ofertă LIMITATĂ! Cumpără Viagra acum cu 90% REDUCERE!",
    "Ai fost selectat pentru un PREMIU CASH de 5000 EUR! Click aici!",
    "FREE MONEY! Win big prizes! Click now!",
    "Câștigă bani rapid de acasă! Fără investiție! Click aici!",
    "URGENT: Contul tău a fost blocat. Click pentru a reactiva ACUM!",
    "Congratulations! You won a FREE iPhone! Claim now!",
    "Make money fast! Work from home! No experience needed!",
    "Slăbește 10 kg în 7 zile! Pastile MAGICE! Comandă acum!",
    "CASINO ONLINE - Bonus 1000 EUR la prima depunere! Joacă acum!",
    "Credit rapid fără acte! Aprobat în 5 minute! Click aici!",
    "Winner! You have been selected for cash prize! Act now!",
    "Cumpără followeri Instagram! 10000 followeri doar 50 EUR!",
    "OFERTĂ ȘOCANTĂ! Rolex original doar 99 EUR! Stock limitat!",
    "Câștigă bani din reclame! 500 EUR/zi garantat!",
]

ham_messages = [
    "Bună! Ne vedem mâine la cafea?",
    "Ți-am trimis raportul pe email. Verifică te rog.",
    "Meetingul de astăzi este amânat pentru mâine la 10:00.",
    "Hi, how are you doing today?",
    "Ai terminat proiectul la statistică? Să discutăm.",
    "Documentația pentru cursul de AI este disponibilă pe platformă.",
    "Can we schedule a call for tomorrow afternoon?",
    "Mulțumesc pentru ajutor cu tema! A fost foarte util.",
    "Conferința despre Machine Learning este pe 15 martie.",
    "Your order has been shipped. Tracking number: ABC123.",
    "Bună seara! Ai primit notițele de la curs?",
    "Team meeting at 3 PM in conference room B.",
    "Ar fi bine să ne pregătim pentru examen împreună.",
    "The project deadline has been extended by one week.",
    "Mulțumesc pentru recomandarea de carte! O să o citesc.",
]

# Crearea dataframe-ului
data = pd.DataFrame({
    'message': spam_messages + ham_messages,
    'label': ['spam'] * len(spam_messages) + ['ham'] * len(ham_messages)
})

# Shuffle dataset
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"📊 Dataset creat: {len(data)} mesaje")
print(f"   - Spam: {sum(data['label'] == 'spam')} mesaje")
print(f"   - Ham: {sum(data['label'] == 'ham')} mesaje")
print("\n🔍 Primele 5 exemple:")
data.head()

## 3️⃣ Preprocesarea Textului

Transformăm textul în format utilizabil pentru algoritm:

In [ ]:
def preprocess_text(text):
    """
    Preprocesare text:
    1. Convertire la lowercase
    2. Eliminare caractere speciale
    3. Tokenizare în cuvinte
    """
    # Lowercase
    text = text.lower()
    
    # Păstrează doar litere și spații
    text = re.sub(r'[^a-zăâîșț\s]', '', text)
    
    # Tokenizare
    tokens = text.split()
    
    return tokens

# Aplicare preprocesare
data['tokens'] = data['message'].apply(preprocess_text)

# Exemplu de preprocesare
print("📝 Exemplu de preprocesare:")
print(f"\nOriginal: {data.iloc[0]['message']}")
print(f"Tokenizat: {data.iloc[0]['tokens']}")
print(f"Label: {data.iloc[0]['label']}")

## 4️⃣ Implementare Naive Bayes de la Zero

### Algoritmul pas cu pas:

1. **Training:** Calculează probabilități pentru fiecare cuvânt în spam vs ham
2. **Prediction:** Aplică Teorema Bayes pentru mesaj nou
3. **Smoothing:** Adaugă Laplace smoothing pentru cuvinte noi

In [ ]:
class NaiveBayesClassifier:
    """
    Implementare Naive Bayes de la zero pentru clasificarea spam
    """
    
    def __init__(self, alpha=1.0):
        """
        Args:
            alpha: Parametru pentru Laplace smoothing (evită probabilități = 0)
        """
        self.alpha = alpha
        self.class_probs = {}  # P(spam) și P(ham)
        self.word_probs = {}   # P(word|spam) și P(word|ham)
        self.vocabulary = set()
        
    def fit(self, messages, labels):
        """
        Antrenare: Calculează probabilitățile din datele de training
        """
        # 1. Calculează probabilitățile claselor: P(spam) și P(ham)
        total_messages = len(labels)
        self.class_probs['spam'] = sum(labels == 'spam') / total_messages
        self.class_probs['ham'] = sum(labels == 'ham') / total_messages
        
        # 2. Construiește vocabularul și numără apariții cuvinte
        word_counts = {'spam': Counter(), 'ham': Counter()}
        
        for message, label in zip(messages, labels):
            for word in message:
                self.vocabulary.add(word)
                word_counts[label][word] += 1
        
        # 3. Calculează P(word|class) cu Laplace smoothing
        vocab_size = len(self.vocabulary)
        
        for class_name in ['spam', 'ham']:
            self.word_probs[class_name] = {}
            total_words = sum(word_counts[class_name].values())
            
            for word in self.vocabulary:
                count = word_counts[class_name][word]
                # Laplace smoothing: (count + alpha) / (total + alpha * vocab_size)
                self.word_probs[class_name][word] = \
                    (count + self.alpha) / (total_words + self.alpha * vocab_size)
        
        print(f"✅ Model antrenat!")
        print(f"   - Vocabular: {vocab_size} cuvinte unice")
        print(f"   - P(spam) = {self.class_probs['spam']:.3f}")
        print(f"   - P(ham) = {self.class_probs['ham']:.3f}")
    
    def predict_proba(self, message):
        """
        Calculează probabilitățile P(spam|message) și P(ham|message)
        """
        # Calculează log probabilități (previne underflow numeric)
        log_probs = {}
        
        for class_name in ['spam', 'ham']:
            # Start cu log(P(class))
            log_prob = np.log(self.class_probs[class_name])
            
            # Adaugă log(P(word|class)) pentru fiecare cuvânt
            for word in message:
                if word in self.vocabulary:
                    log_prob += np.log(self.word_probs[class_name][word])
            
            log_probs[class_name] = log_prob
        
        # Convertește înapoi din log space (normalizare)
        max_log_prob = max(log_probs.values())
        probs = {}
        
        for class_name in ['spam', 'ham']:
            probs[class_name] = np.exp(log_probs[class_name] - max_log_prob)
        
        # Normalizare la sumă = 1
        total = sum(probs.values())
        probs = {k: v/total for k, v in probs.items()}
        
        return probs
    
    def predict(self, message):
        """
        Predicție clasă (spam sau ham)
        """
        probs = self.predict_proba(message)
        return 'spam' if probs['spam'] > probs['ham'] else 'ham'
    
    def get_top_words(self, class_name, n=10):
        """
        Returnează cele mai indicative n cuvinte pentru o clasă
        """
        word_scores = {}
        
        for word in self.vocabulary:
            # Calculează log ratio: log(P(word|class) / P(word|other_class))
            other_class = 'ham' if class_name == 'spam' else 'spam'
            ratio = self.word_probs[class_name][word] / self.word_probs[other_class][word]
            word_scores[word] = np.log(ratio)
        
        # Sortează și returnează top N
        top_words = sorted(word_scores.items(), key=lambda x: x[1], reverse=True)[:n]
        return top_words

print("✅ Clasa NaiveBayesClassifier definită!")

## 5️⃣ Antrenare și Testare Model

In [ ]:
# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    data['tokens'].values, 
    data['label'].values,
    test_size=0.25,
    random_state=42,
    stratify=data['label']
)

print(f"📊 Split dataset:")
print(f"   Training: {len(X_train)} mesaje")
print(f"   Testing: {len(X_test)} mesaje")

# Antrenare model
print("\n🔧 Antrenare model...")
nb_classifier = NaiveBayesClassifier(alpha=1.0)
nb_classifier.fit(X_train, y_train)

## 6️⃣ Evaluare Performanță

In [ ]:
# Predicții pe setul de test
y_pred = [nb_classifier.predict(message) for message in X_test]
y_pred_proba = [nb_classifier.predict_proba(message)['spam'] for message in X_test]

# Acuratețe
accuracy = np.mean(np.array(y_pred) == y_test)
print(f"🎯 Acuratețe: {accuracy * 100:.2f}%")

# Raport de clasificare detaliat
print("\n📊 Raport de Clasificare:")
print(classification_report(y_test, y_pred, target_names=['ham', 'spam']))

## 7️⃣ Vizualizări Interactive

In [ ]:
# Configurare plot
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('📊 Analiză Completă Naive Bayes Spam Classifier', fontsize=16, fontweight='bold')

# 1. Matricea de Confuzie
cm = confusion_matrix(y_test, y_pred, labels=['ham', 'spam'])
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0, 0],
            xticklabels=['Ham', 'Spam'], yticklabels=['Ham', 'Spam'])
axes[0, 0].set_title('Matricea de Confuzie', fontsize=14, fontweight='bold')
axes[0, 0].set_ylabel('Adevărat')
axes[0, 0].set_xlabel('Prezis')

# 2. Distribuția Probabilităților
spam_probs = [p for p, label in zip(y_pred_proba, y_test) if label == 'spam']
ham_probs = [p for p, label in zip(y_pred_proba, y_test) if label == 'ham']

axes[0, 1].hist(spam_probs, bins=10, alpha=0.6, label='Spam Real', color='red', edgecolor='black')
axes[0, 1].hist(ham_probs, bins=10, alpha=0.6, label='Ham Real', color='green', edgecolor='black')
axes[0, 1].set_xlabel('P(Spam)', fontsize=12)
axes[0, 1].set_ylabel('Frecvență', fontsize=12)
axes[0, 1].set_title('Distribuția Probabilităților de Spam', fontsize=14, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. Top Cuvinte Spam
top_spam_words = nb_classifier.get_top_words('spam', n=10)
words_spam = [w[0] for w in top_spam_words]
scores_spam = [w[1] for w in top_spam_words]

axes[1, 0].barh(words_spam, scores_spam, color='crimson', edgecolor='black')
axes[1, 0].set_xlabel('Log Likelihood Ratio', fontsize=12)
axes[1, 0].set_title('Top 10 Cuvinte Indicative pentru SPAM', fontsize=14, fontweight='bold')
axes[1, 0].invert_yaxis()
axes[1, 0].grid(True, alpha=0.3, axis='x')

# 4. Top Cuvinte Ham
top_ham_words = nb_classifier.get_top_words('ham', n=10)
words_ham = [w[0] for w in top_ham_words]
scores_ham = [w[1] for w in top_ham_words]

axes[1, 1].barh(words_ham, scores_ham, color='mediumseagreen', edgecolor='black')
axes[1, 1].set_xlabel('Log Likelihood Ratio', fontsize=12)
axes[1, 1].set_title('Top 10 Cuvinte Indicative pentru HAM', fontsize=14, fontweight='bold')
axes[1, 1].invert_yaxis()
axes[1, 1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("\n✅ Vizualizări generate cu succes!")

## 8️⃣ Curba ROC și AUC Score

In [ ]:
# Calculează curba ROC
y_test_binary = (y_test == 'spam').astype(int)
fpr, tpr, thresholds = roc_curve(y_test_binary, y_pred_proba)
roc_auc = auc(fpr, tpr)

# Plot ROC
plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, color='darkorange', lw=3, label=f'ROC Curve (AUC = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=14)
plt.ylabel('True Positive Rate', fontsize=14)
plt.title('📈 Curba ROC - Naive Bayes Spam Classifier', fontsize=16, fontweight='bold')
plt.legend(loc="lower right", fontsize=12)
plt.grid(True, alpha=0.3)
plt.show()

print(f"\n🎯 AUC Score: {roc_auc:.4f}")
print("   → 0.5 = clasificare aleatorie")
print("   → 1.0 = clasificare perfectă")

## 9️⃣ Test Interactiv pe Mesaje Noi

In [ ]:
def analyze_message(message):
    """
    Analizează un mesaj nou și afișează detalii despre predicție
    """
    print("="*80)
    print(f"📧 Mesaj: {message}")
    print("="*80)
    
    # Preprocesare
    tokens = preprocess_text(message)
    print(f"\n🔤 Tokens: {tokens}")
    
    # Predicție
    probs = nb_classifier.predict_proba(tokens)
    prediction = nb_classifier.predict(tokens)
    
    # Afișare rezultate
    print(f"\n📊 Probabilități:")
    print(f"   P(Spam|mesaj) = {probs['spam']:.4f} ({probs['spam']*100:.2f}%)")
    print(f"   P(Ham|mesaj)  = {probs['ham']:.4f} ({probs['ham']*100:.2f}%)")
    
    print(f"\n🎯 Predicție: {'🚨 SPAM' if prediction == 'spam' else '✅ HAM (legitim)'}")
    
    # Vizualizare probabilități
    plt.figure(figsize=(8, 4))
    colors = ['crimson' if prediction == 'spam' else 'mediumseagreen', 
              'lightcoral' if prediction == 'spam' else 'lightgreen']
    plt.bar(['Spam', 'Ham'], [probs['spam'], probs['ham']], 
            color=colors, edgecolor='black', linewidth=2)
    plt.ylabel('Probabilitate', fontsize=12)
    plt.title(f"Predicție: {'SPAM' if prediction == 'spam' else 'HAM'}", 
              fontsize=14, fontweight='bold')
    plt.ylim([0, 1])
    plt.grid(True, alpha=0.3, axis='y')
    
    # Adaugă valori pe bare
    for i, (cat, prob) in enumerate(zip(['Spam', 'Ham'], [probs['spam'], probs['ham']])):
        plt.text(i, prob + 0.02, f'{prob:.3f}', ha='center', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    print("\n")

# Testare pe mesaje noi
test_messages = [
    "FELICITĂRI! Ai câștigat un iPhone 15 GRATUIT! Click ACUM!",
    "Bună! Ne vedem la meeting mâine la ora 10?",
    "URGENT: Contul tău a expirat. Reactivează ACUM pentru bonus 1000 EUR!",
    "Raportul de statistică este gata. L-am încărcat pe drive.",
]

for msg in test_messages:
    analyze_message(msg)

## 🔟 Comparație cu Sklearn Implementation

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Creează pipeline sklearn
sklearn_pipeline = Pipeline([
    ('vectorizer', CountVectorizer(lowercase=True, token_pattern=r'[a-zăâîșț]+')),
    ('classifier', MultinomialNB(alpha=1.0))
])

# Antrenare
X_train_text = [' '.join(tokens) for tokens in X_train]
X_test_text = [' '.join(tokens) for tokens in X_test]

sklearn_pipeline.fit(X_train_text, y_train)
y_pred_sklearn = sklearn_pipeline.predict(X_test_text)
accuracy_sklearn = np.mean(y_pred_sklearn == y_test)

# Comparație
print("📊 Comparație Performanță:\n")
print(f"{'Model':<30} {'Acuratețe':<15}")
print("="*45)
print(f"{'Implementare Custom':<30} {accuracy * 100:>13.2f}%")
print(f"{'Sklearn MultinomialNB':<30} {accuracy_sklearn * 100:>13.2f}%")
print("="*45)
print(f"\n✅ Diferență: {abs(accuracy - accuracy_sklearn) * 100:.2f}%")
print("\n💡 Implementarea noastră oferă rezultate similare cu sklearn!")

## 📚 Concluzii și Insights

### Ce am învățat:

1. **Teoria în Practică:**
   - Teorema Bayes nu este doar matematică abstractă - este fundamentul unor sisteme AI practice
   - Asumpția de independență ("naive") funcționează surprinzător de bine în practică

2. **Importanța Smoothing-ului:**
   - Laplace smoothing previne probabilități = 0 pentru cuvinte noi
   - Parametrul α controlează cât de mult "încredere" dăm datelor vs distribuției uniforme

3. **Performanță:**
   - Naive Bayes este rapid și eficient
   - Funcționează bine chiar cu dataset-uri mici
   - Scalează excelent la date mari (complexitate O(n))

4. **Limitări:**
   - Asumpția de independență este adesea violată în realitate
   - Nu captează relații contextuale complexe
   - Pentru texte complexe, modele mai avansate (LSTM, Transformers) pot performa mai bine

### Aplicații Practice:
- ✉️ Filtrare spam email
- 💬 Analiza sentimentului
- 📰 Clasificare documente
- 🏥 Diagnostic medical (cu date categorice)
- 🔍 Sisteme de recomandare

### Next Steps:
1. Extindere cu TF-IDF în loc de simple count
2. Experimentare cu n-grams (bigrams, trigrams)
3. Combinare cu alte algoritmi (ensemble methods)
4. Deployment ca API web cu Flask/FastAPI

---

**📖 Referințe:**
- Bishop, C. M. (2006). Pattern Recognition and Machine Learning
- Manning, C. D., Raghavan, P., & Schütze, H. (2008). Introduction to Information Retrieval
- Russell, S., & Norvig, P. (2020). Artificial Intelligence: A Modern Approach